In [89]:
import pandas as pd
import plotly.express as px
from unidecode import unidecode
import re
import numpy as np
from os import listdir
from collections import Counter

In [90]:
PATH_RAW = r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\raw'
PATH_PROCESSED = r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\processed'

In [91]:
def cleaning_tweets(data, language='pt'):
    dataset = data.copy()
    dataset = dataset[dataset['language'] == language]
    dataset.drop_duplicates('tweet', inplace=True)
    dataset.drop_duplicates('username', inplace=True)
    dataset['hashtags'] = [", ".join(y for y in re.findall('\B#\w\w+', x)) for x in dataset['tweet']]
    dataset = dataset[['date','tweet','hashtags']]
    dataset

    return dataset

In [92]:
for f in listdir(PATH_RAW):
    dataset = pd.read_csv(PATH_RAW + "\\" + f)
    dataset = cleaning_tweets(dataset)
    dataset.to_csv(PATH_PROCESSED + "\\" + f)

In [93]:
dataset = pd.DataFrame()

for f in listdir(PATH_PROCESSED):
    df = pd.read_csv(PATH_PROCESSED + "\\" + f)
    dataset = pd.concat([dataset, df])
    
dataset = dataset.drop_duplicates('tweet')

In [94]:
dataset['hashtags'] = dataset['hashtags'].fillna('#nan')
dataset['hashtags'] = dataset['hashtags'].apply(lambda x: re.sub("#", '', x))
dataset['hashtags'] = dataset['hashtags'].apply(lambda x: x.lower())
dataset['hashtags'] = [unidecode(x) for x in dataset['hashtags']]

In [95]:
dataset['hashtags'] = [x.split(',') for x in dataset['hashtags']]

In [96]:
arr = []
for x in dataset['hashtags']:
    arr.extend(x)

In [97]:
arr = pd.DataFrame(arr)

In [98]:
arr.value_counts().head(10)

setembroamarelo          15584
nan                       2904
 setembroamarelo          2131
 saudemental              1727
saudemental               1686
sessaodeterapia            568
eumeimporto                525
suicidiomateessaideia      524
suavidaimporta             499
depressaonaoefrescura      444
dtype: int64

In [99]:
dataset['tweet'] = dataset['tweet'].apply(lambda x: x.lower())
dataset['tweet'] = [unidecode(x) for x in dataset['tweet']]
dataset['tweet'] = [re.sub('\B#\w\w+', "",x) for x in dataset['tweet']]
dataset['tweet'] = [re.sub('\B@\w\w+', "",x) for x in dataset['tweet']]

In [104]:
dataset = dataset[['date', 'tweet']].reset_index().drop(columns='index')

In [108]:
dataset = dataset.sort_values('date').set_index('date')

In [115]:
dataset.drop_duplicates('tweet', inplace=True)

In [117]:
dataset.to_csv(r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\cleaned\data.csv')